In [1]:
import pandas as pd
from online_cp import PluginMartingale
from online_cp import NearestNeighboursPredictionMachine, RidgePredictionMachine
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
rnd_gen = np.random.default_rng(2025)
from copy import deepcopy
from sklearn.preprocessing import minmax_scale
MACHINE_EPSILON = lambda x: np.abs(x) * np.finfo(np.float64).eps

# Need to figure out normalisation. K-nn is not all that good without it...

It would also be very nice to figure out how to make it Mondrian...

Easiest should be to normalise the data as we go along, and use learn_initial_training_set each time. It will slow down things a bit, but it would work in principle.

In [2]:
data = pd.read_csv('export.csv', parse_dates=True, index_col='Time')
data = data.asfreq('h')
data = data.dropna()

Winter_train = data[:4308]
Winter_test = data[4308:]

D = 24

In [3]:
initial_training_set_size = D + Winter_train.shape[0]
knonw_initially = data[:initial_training_set_size]
future = data[initial_training_set_size:]

X_initial_train = knonw_initially[:-D]
y_initial_train = knonw_initially[D:].demand

In [4]:
learn_cols = ['demand', f'temperature_{D}', f'Hour_{D}', f'Working_day_{D}']
pred_cols = ['demand', f'temperature_forecast', f'Hour_{D}', f'Working_day_{D}']

X_train = X_initial_train[learn_cols]
y_train = y_initial_train

# cps = NearestNeighboursPredictionMachine(k=neighbours)
cps = RidgePredictionMachine(autotune=True)
cps.learn_initial_training_set(
    X=X_train[learn_cols].values,
    y=y_train.values
)
cpd = cps.predict_cpd(X_initial_train.iloc[-1][pred_cols].values)

INIT


In [5]:
self = cpd

# self.y_vals = np.array(sorted([-np.inf, np.inf] + self.C[1: -1].tolist() + (self.C[1: -1] + MACHINE_EPSILON(self.C[1: -1])).tolist() + (self.C[1: -1] - MACHINE_EPSILON(self.C[1: -1])).tolist()))
# self.L = np.array([self.__call__(y, 0) for y in self.y_vals])
# self.U = np.array([self.__call__(y, 1) for y in self.y_vals])

In [10]:
p = 1
tau = 0.442142334
self.Y[np.where((1 - tau) * self.L + tau * self.U >= p)[0].min()]

inf

In [12]:
for _ in range(100):
    self.quantile(p, tau)

In [ ]:
hourly = False

neighbours = 101

autotune=True

normalisation = False

pred_cols = ['demand', f'temperature_forecast', f'Hour_{D}', f'Working_day_{D}']
learn_cols = ['demand', f'temperature_{D}', f'Hour_{D}', f'Working_day_{D}']
Hours = data.index.hour.unique().to_numpy()

if normalisation:

    if hourly:
        X_train_dict = {hour: X_initial_train.loc[X_initial_train.index.hour==hour][learn_cols] for hour in Hours}
        y_train_dict = {hour: y_initial_train.loc[X_initial_train.index.hour==hour].values for hour in Hours}

        # hourly_CPS = {hour: NearestNeighboursPredictionMachine(k=neighbours) for hour in Hours}
        hourly_CPS = {hour: RidgePredictionMachine(autotune=autotune) for hour in Hours}

        [hourly_CPS[hour].learn_initial_training_set(
                X=minmax_scale(X_train_dict[hour]),
                y=y_train_dict[hour]
                ) for hour in Hours]

    else:
        X_train = X_initial_train[learn_cols]
        y_train = y_initial_train

        # cps = NearestNeighboursPredictionMachine(k=neighbours)
        cps = RidgePredictionMachine(autotune=autotune)
        cps.learn_initial_training_set(
            X=minmax_scale(X_train),
            y=y_train.values
        )

    martingale = PluginMartingale(warnings=False, window_size='adaptive')
    calibration_martingale = PluginMartingale(warnings=False)

    res = np.zeros(shape=(future.shape[0]-D, 4))
    p_values = np.zeros(shape=(future.shape[0]-D,))
    protected_p_values = np.zeros(shape=(future.shape[0]-D,))
    p_values_cat = {i: [] for i in Hours}
    protected_p_values_cat = {i: [] for i in Hours}

    dists = {}
    protected_dists = {}

    limit = 40

    j = 0
    for i, obj in tqdm(future.iterrows(), total=future.shape[0], desc=f'Decision protocol {D}h (normalised data)'):
        tau = rnd_gen.uniform(0, 1)

        # This is all we know now
        known = data[data.index <= i]

        if hourly:
            cps = hourly_CPS[i.hour]   
            X_train = X_train_dict[i.hour]
            y_train = y_train_dict[i.hour]

        # Reality outputs object
        x = obj[pred_cols].values

        # And we scale it
        X_scale = np.vstack([X_train, x])
        x = minmax_scale(X_scale)[-1]

        # Forecaster outputs conformal predictive distribution
        cpd = cps.predict_cpd(x=x)

        protected_cpd = lambda y, tau: martingale.B_n(cpd(y, tau)) # FIXME This object changes after it has been saved...

        pred = cpd(limit, tau)
        protected_pred = protected_cpd(limit, tau)

        # Reality outputs label (the demand now) for the object x_{t-24}
        y = obj.demand
        x_minus_24 = known.loc[i - pd.Timedelta(D, 'h')][learn_cols].values
        # And we scale it
        X_train = np.vstack([X_train, x_minus_24])
        y_train = np.append(y_train, y)
        x_minus_24 = minmax_scale(X_scale)[-1]

        # Learn new example (relearn everything)
        cat_learn = (i - pd.Timedelta(D, 'h')).hour
        if hourly:
            hourly_CPS[cat_learn].learn_initial_training_set(X=minmax_scale(X_train), y=y_train)
        else:
            cps.learn_initial_training_set(X=minmax_scale(X_train), y=y_train)

        # hourly_CPS[cat_learn].learn_one(x_minus_24, y)
        # cps.learn_one(x_minus_24, y)

        if j < future.shape[0] - 24:
            y_plus_24 = future.loc[i + pd.Timedelta(D, 'h')].demand
            res[j, 0] = pred
            res[j, 1] = protected_pred
            res[j, 2] = y_plus_24
            res[j, 3] = tau
            dists[j] = cpd
            protected_dists[j] = deepcopy(protected_cpd)

            p = cpd(y_plus_24, tau)
            protected_p = protected_cpd(y_plus_24, tau)
            p_values[j] = p
            p_values_cat[cat_learn].append(p)
            protected_p_values[j] = protected_p
            protected_p_values_cat[cat_learn].append(protected_p)
            martingale.update_martingale_value(p)
            calibration_martingale.update_martingale_value(protected_p)
        j += 1

else:

    if hourly:
        # hourly_CPS = {hour: NearestNeighboursPredictionMachine(k=neighbours) for hour in Hours}
        hourly_CPS = {hour: RidgePredictionMachine(autotune=autotune) for hour in Hours}

        [hourly_CPS[hour].learn_initial_training_set(
                X=X_initial_train.loc[X_initial_train.index.hour==hour][learn_cols].values,
                y=y_initial_train.loc[X_initial_train.index.hour==hour].values
                ) for hour in Hours]
    else:
        # cps = NearestNeighboursPredictionMachine(k=neighbours)
        cps = RidgePredictionMachine(autotune=autotune)
        cps.learn_initial_training_set(
            X=X_initial_train[learn_cols].values,
            y=y_initial_train.values
        )

    martingale = PluginMartingale(warnings=False, window_size='adaptive')
    calibration_martingale = PluginMartingale(warnings=False)

    res = np.zeros(shape=(future.shape[0]-D, 4))
    p_values = np.zeros(shape=(future.shape[0]-D,))
    protected_p_values = np.zeros(shape=(future.shape[0]-D,))
    p_values_cat = {i: [] for i in Hours}
    protected_p_values_cat = {i: [] for i in Hours}

    dists = {}
    protected_dists = {}

    limit = 40

    j = 0
    for i, obj in tqdm(future.iterrows(), total=future.shape[0], desc=f'Decision protocol {D}h (raw data)'):
        tau = rnd_gen.uniform(0, 1)

        # This is all we know now
        known = data[data.index <= i]

        if hourly:
            cps = hourly_CPS[i.hour]   

        # Reality outputs object
        x = obj[pred_cols].values

        # Forecaster outputs conformal predictive distribution
        cpd = cps.predict_cpd(x=x)

        protected_cpd = lambda y, tau: martingale.B_n(cpd(y, tau)) # FIXME This object changes after it has been saved...

        pred = cpd(limit, tau)
        protected_pred = protected_cpd(limit, tau)

        # Reality outputs label (the demand now) for the object x_{t-24}
        y = obj.demand
        x_minus_24 = known.loc[i - pd.Timedelta(D, 'h')][learn_cols].values

        # Learn new example
        cat_learn = (i - pd.Timedelta(D, 'h')).hour
        if hourly:
            hourly_CPS[cat_learn].learn_one(x_minus_24, y)
        else:
            cps.learn_one(x_minus_24, y)

        if j < future.shape[0] - 24:
            y_plus_24 = future.loc[i + pd.Timedelta(D, 'h')].demand
            res[j, 0] = pred
            res[j, 1] = protected_pred
            res[j, 2] = y_plus_24
            res[j, 3] = tau
            dists[j] = cpd
            protected_dists[j] = deepcopy(protected_cpd)

            p = cpd(y_plus_24, tau)
            protected_p = protected_cpd(y_plus_24, tau)
            p_values[j] = p
            p_values_cat[cat_learn].append(p)
            protected_p_values[j] = protected_p
            protected_p_values_cat[cat_learn].append(protected_p)
            martingale.update_martingale_value(p)
            calibration_martingale.update_martingale_value(protected_p)
        j += 1

In [ ]:
cpd = dists[144]
cpd.plot()

In [6]:
# cpd = dists[144]

# yrange = np.linspace(0, 50, num=1000, endpoint=True)
# plt.plot(
#     yrange,
#     [cpd(y=y, tau=0) for y in yrange],
#     label=r'$\Pi(y, 0)$'
# )
# plt.plot(
#     yrange,
#     [cpd(y=y, tau=1) for y in yrange],
#     label=r'$\Pi(y, 1)$'
# )

# plt.fill_between(
#     yrange,
#     [cpd(y=y, tau=0) for y in yrange],
#     [cpd(y=y, tau=1) for y in yrange],
#     alpha=0.7,
#     color='C2'
# )
# plt.legend()

In [ ]:
from sklearn.metrics import log_loss
base_log_loss = log_loss(y_true=(res[:, 2] <= limit).astype(int), y_pred=res[:, 0])
protected_log_loss = log_loss(y_true=(res[:, 2] <= limit).astype(int), y_pred=res[:, 1])

from sklearn.metrics import brier_score_loss
base_brier_score_loss = brier_score_loss(y_true=(res[:, 2] <= limit).astype(int), y_proba=res[:, 0])
protected_brier_score_loss = brier_score_loss(y_true=(res[:, 2] <= limit).astype(int), y_proba=res[:, 1])

from sklearn.metrics import roc_auc_score
base_roc_auc_score = roc_auc_score(y_true=(res[:, 2] <= limit).astype(int), y_score=res[:, 0])
protected_roc_auc_score = roc_auc_score(y_true=(res[:, 2] <= limit).astype(int), y_score=res[:, 1])

from sklearn.metrics import average_precision_score
base_average_precision_score = average_precision_score(y_true=(res[:, 2] <= limit).astype(int), y_score=res[:, 0])
protected_average_precision_score = average_precision_score(y_true=(res[:, 2] <= limit).astype(int), y_score=res[:, 1])

losses = np.array(
    [
        [base_log_loss, base_brier_score_loss, base_roc_auc_score, base_average_precision_score],
        [protected_log_loss, protected_brier_score_loss, protected_roc_auc_score, protected_average_precision_score]
    ]
)
df_losses = pd.DataFrame(data=losses, index=['base', 'protected'], columns=['log_loss', 'brier_score', 'roc_auc', 'pr_auc'])
df_losses

In [ ]:
confidence = 0.95
base_warnings = (res[:, 0] < confidence).sum()
protected_warnings = (res[:, 1] < confidence).sum()
print(f'Base warnigns at confidence {confidence}: {base_warnings}')
print(f'Protected warnigns at confidence {confidence}: {protected_warnings}')

In [ ]:
plt.plot(res[:, 0])
plt.plot(res[:, 1], alpha=0.5)
plt.axhline(0.95, linestyle='--')

In [ ]:
h = 13
plt.hist(p_values_cat[h], density=True)
plt.hist(protected_p_values_cat[h], density=True, alpha=0.7)

In [ ]:
plt.plot(martingale.log10_martingale_values)
plt.plot(calibration_martingale.log10_martingale_values)

In [ ]:
uppers = np.array([dists[i].quantile(0.95, tau) for i, tau in zip(dists.keys(), res[:, 3])])
(uppers < res[:, 2]).mean()

In [ ]:
uppers

In [15]:
# Quantiles = {}

# for quantile in tqdm(np.linspace(0.01, 1, 100, endpoint=False)):

#     quantile = np.round(quantile, 2)

#     uppers = np.array([dists[i].quantile(quantile, tau) for i, tau in zip(dists.keys(), res[:, 3])])

#     Quantiles[quantile] = uppers

In [ ]:
post_martingale = PluginMartingale(warnings=False)
for p in p_values:
    post_martingale.update_martingale_value(p)

prange = np.linspace(0, 1, 1000)
plt.plot(
    prange,
    post_martingale.B_n(prange),
    label='base'
)
plt.plot(
    prange,
    calibration_martingale.B_n(prange),
    label='protected'
)
plt.plot(
    prange,
    prange,
    label='perfect',
    linestyle='--'
)
plt.legend()

In [ ]:
plt.hist(p_values, density=True)
plt.hist(protected_p_values, density=True, alpha=0.7)

# How to use CPD for decision making?
Let $\Pi$ be the CPD, and say we have a decision space $D$. Then we could take
$$\hat{d} = \text{argmax}_d \int_{-\infty}^{\infty}U(d,y)\Pi(dy)$$
as the decision. Now we just have to figure out how to compute this quantity for a decision, and then figure out if it is possible to do it for all decisions...

In [18]:
def utility(flex, limit, demand, flex_cost=0.1, overdraw_cost=1):
    '''
    Simple utility function that takes into account the possibility to buy too little flex.
    NOTE: This is unrealistic in that we can not buy a real number of flex in practice.
    '''
    effective_demand = demand - flex # Are we above despide having flex?
    effective_above = effective_demand > limit 
    above = demand > limit # Would we have been above if we did not buy flex?
    if effective_above: # Even with flex, we are above the limit
        u = - overdraw_cost * (effective_demand - limit) - flex_cost * flex
    elif above: # We would have been over the limit if we had not had flex.
        u = overdraw_cost * (demand - limit) - flex_cost * flex
    else: # We are below no matter what.
        u = - flex_cost * flex
    return float(u)

def exp_util(cpd, d, tau=1):
    jump_points = np.unique(cpd.Y[1:-1])
    Delta_Q = np.array([utility(d, limit, y) * (cpd(y=y + MACHINE_EPSILON(y), tau=tau) - cpd(y=y - MACHINE_EPSILON(y), tau=tau)) for y in jump_points])
    return jump_points @ Delta_Q

def expectation(cpd, tau=1):
    jump_points = np.unique(cpd.Y[1:-1])
    Delta_Q = np.array([(cpd(y=y + MACHINE_EPSILON(y), tau=tau) - cpd(y=y - MACHINE_EPSILON(y), tau=tau)) for y in jump_points])
    return jump_points @ Delta_Q

In [ ]:
np.where(res[:, 2] > 40)

In [ ]:
i = 154
cpd = dists[i]
plt.step(cpd.Y[:], (1 - tau) * cpd.L + tau * cpd.U, label=r'$\Pi(y, \tau)$')
res[i, 2], cpd(limit, 0), res[i, 0]
plt.axvline(res[i, 2], color='red', linestyle='--', label='truth')
plt.axvline(cpd.quantile(0.9, tau=tau), color='C1', label='0.9 quantile')
plt.axvline(expectation(cpd, tau), linestyle='--', label='expectation', color='C2')
plt.legend()

In [ ]:
qrange = np.linspace(0.01, 1, endpoint=False, num=99)
plt.plot(
    qrange,
    [cpd.quantile(q, tau) for q in qrange],
    label='quantile function'
)
plt.axhline(res[i, 2], linestyle='--', color='red')

## This maximisation of expected utility is quite crude. 
A better way may be to fit a CPS to a decision, as in the working paper. However, this is computatinally expensive, and absolutely requires discrete decisions.

Since we are anyway going to need to discretise the decision space, perhaps this is a reasonable way to go. Perhaps we need to speak to the experts?

In [ ]:
from scipy.optimize import minimize_scalar

tau = rnd_gen.uniform(0, 1)
d = minimize_scalar(lambda x: -exp_util(cpd, x, tau), bounds=(0, 10)).x
print()
print(f'Decicion: {d}')
print(f'Optimal decision: {max(res[i, 2] - limit, 0)}')
print(f'Expected utility: {exp_util(cpd, d, tau)}')
print(f'Utility {utility(d, limit, res[i, 2])}')
print()

In [ ]:
drange = np.linspace(0, 10, num=1000, endpoint=True)
plt.plot(
    drange,
    [exp_util(cpd, d, tau) for d in drange]
)
plt.plot(
    drange,
    [utility(d, limit, res[i, 2]) for d in drange]
)
plt.axvline(max(res[i, 2] - limit, 0), linestyle='--', color='red', label='optimal decision')

In [ ]:
exp_util(cpd, d, tau)

In [ ]:
utility(d, limit, res[i, 2])

# Yet another alternative is to simply buy if the predicted probability of being below is "too small"

In [ ]:
cpd.Y

In [ ]:
cpd(limit, tau), cpd.quantile(0.9, tau)

# Another decision idea:
Comming back to an early idea; if we use CPS to warn, which it seems to do quite well, we can then train a forecaster on data that historically exceeded the limit, thus disregarding all old examples that are small.

As we can see, in most cases, the demand 24 hours before was quite high in previous examples.

In [ ]:
plt.plot(data[data.demand_24 > limit].demand.values)

What about; if we warn, train a new CPD, based on the following training set: All examples in the past whose label exceed the limit.